### Imports and Notebook setup

In [ ]:
# library with nice formating
!pip install rich

In [ ]:
# exporti train
from tqdm.notebook import tqdm

In [ ]:
# default_exp resnet
import os

if os.environ.get('KAGGLE_KERNEL_RUN_TYPE', 'Localhost') == 'Losthost':
    from nbdev import *

In [ ]:
# exporti train
import rich

from collections import OrderedDict

In [ ]:
# exporti
import numpy as np
import pandas as pd

In [ ]:
# exporti
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# exporti train
from torch.utils.data import dataset, dataloader, random_split

from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor, Lambda, Compose

In [ ]:
# exporti train
dataset = CIFAR10(root="/kaggle/working/data/",
                        download=True,
                        train=True, transform=ToTensor())
test_dataset = CIFAR10(root="/kaggle/working/data/",
                       download=True, train=False,
                       transform=ToTensor())

# create train, valid datasets
valid_size = int(dataset.data.shape[0]*0.1)
train_size = dataset.data.shape[0] - valid_size

train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

In [ ]:
# exporti train
config = OrderedDict()
config['batch_size'] = 128

In [ ]:
# exporti train
train_dataloader = dataloader.DataLoader(train_dataset, batch_size=config['batch_size'])
valid_dataloader = dataloader.DataLoader(valid_dataset, batch_size=config['batch_size'])
test_dataloader = dataloader.DataLoader(test_dataset, batch_size=config['batch_size'])

In [ ]:
# hide
for x, y in train_dataloader:
    rich.print(x.max(), x.min())
    rich.print(y)
    break

### ResNet Block

In [ ]:
# export
class ResNetBlock(nn.Module):
    
    def __init__(self, in_channels: int = 32, num_filters: int = 64):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels,
                               out_channels=num_filters,
                               kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_filters)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=num_filters,
                               out_channels=num_filters, 
                               kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(num_filters)
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        z = self.conv1(x)
        z = self.relu1(z)
        z = self.bn1(z)
        z = self.conv2(z)
        y = self.relu2(z + x)
        return self.bn2(y)

Check that `ResNetBlock` has same input and output sizes

In [ ]:
rblock = ResNetBlock(in_channels=64)
x = torch.randint(0, 100, size=(128, 64, 32, 32), dtype=torch.float32)
y = rblock(x)
assert x.shape == y.shape

`ResNetChangeBlock` implements the ResNet with skip connections when the input and output have different shape

In [ ]:
# export
class ResNetChangeBlock(nn.Module):
    
    def __init__(self, in_channels: int = 32, num_filters: int = 64):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = num_filters
        self.conv1 = nn.Conv2d(in_channels=in_channels,
                               out_channels=num_filters,
                               stride=2,
                               kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(num_filters)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=num_filters,
                               out_channels=num_filters, 
                               kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(num_filters)
        self.relu2 = nn.ReLU()
    
    def forward(self, x):
        n, c, w, h = x.shape
        z = self.conv1(x)
        z = self.relu1(z)
        z = self.bn1(z)
        z = self.conv2(z)
        self.zero_identity = nn.Parameter(
            torch.zeros(n, self.out_channels-self.in_channels,
                        int(w//2), int(h//2)),
            requires_grad=False
        )
        x_expand = F.pad(x[:, :, :int(w//2), :int(h//2)], pad=(
            0, 0, 0, 0, self.out_channels-self.in_channels, 0, 0, 0))
        y = self.relu2(z + x_expand)
        return self.bn2(y)

Check the padding operation works as intended, the pad dimensions are left-to-right last-to-first dimension

In [ ]:
x = torch.ones([128, 16, 16, 16])
y = torch.ones([128, 32, 16, 16])
z = F.pad(x, (0, 0, 0, 0, 16, 0, 0, 0))
assert z.shape == y.shape

Check that `ResNetChangeBlock` has half receptor field

In [ ]:
rblock2 = ResNetChangeBlock(in_channels=64, num_filters=128)
x = torch.randint(0, 100, size=(128, 64, 32, 32), dtype=torch.float32)
y = rblock2(x)
assert y.shape == (128, 128, 16, 16)

`ResNet` with variable `6n+2` number of layers and skip connections `2n` skip connections, Similar to CIFAR10 network from the [resnet paper](https://arxiv.org/abs/1512.03385#).

In [ ]:
# export
class ResNet(nn.Module):
    
    def __init__(self, n: int = 3, num_classes: int = 10):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16,
                               kernel_size=3, padding=1)
        self.resblock1_list = nn.ModuleList([
            ResNetBlock(in_channels=16, num_filters=16) for _ in range(n)
        ])
        self.resblock2_list = nn.ModuleList([
                ResNetChangeBlock(in_channels=16, num_filters=32)
            ] + [
            ResNetBlock(in_channels=32, num_filters=32) for _ in range(n-1)
        ])
        self.resblock3_list = nn.ModuleList([
            ResNetChangeBlock(in_channels=32, num_filters=64)
        ] + [
            ResNetBlock(in_channels=64, num_filters=64) for _ in range(n-1)
        ])
        self.linear1 = nn.Linear(in_features=64,
                                 out_features=num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        for layer1 in self.resblock1_list:
            x = layer1(x)
        for layer2 in self.resblock2_list:
            x = layer2(x)
        for layer3 in self.resblock3_list:
            x = layer3(x)
        x = torch.mean(x, dim=[2, 3])
        x = self.linear1(x)
        return x

Check `ResNet` has last dimension as `num_classes`

In [ ]:
n = 3
num_classes = 10
batch_size = 128
res_net = ResNet(n=n, num_classes=num_classes)
x = torch.randint(0, 100, size=(batch_size, 3, 32, 32), dtype=torch.float32)
y = res_net(x)
assert y.shape == (batch_size, num_classes)

### Define Train and Eval loop

In [ ]:
# exporti train
config['device'] = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device: {config['device']}")

# define the network
config['n'] = 3
num_classes = 10
res_net = ResNet(n=config['n'], num_classes=num_classes)
res_net = res_net.to(config['device'])

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=res_net.parameters())

In [ ]:
rich.print(res_net)

In [ ]:
# exporti train
def train_fn(model: nn.Module, 
             train_dataloader: dataloader.DataLoader,
             loss_fn, optimizer, epoch: int, device: str):
    
    model.train()
    
    for batch_index, (X, y) in tqdm(enumerate(train_dataloader),
                                    total=len(train_dataloader.dataset.indices)//config['batch_size']):
        
        X_data, y_data = X.to(device), y.to(device)
        y_pred = model(X_data)
        loss = loss_fn(y_pred, y_data)
        
        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    loss = loss.item()
    accu = torch.argmax(y_pred, dim=-1) == y_data
    accu = sum(accu)/len(accu)
    accu = accu.item()
    print("loss="+str(loss), end=' ')
    print("accu="+str(accu), end=' ')


In [ ]:
# exporti train
def test_fn(model: nn.Module,
            test_dataloader: dataloader.DataLoader,
            epoch: int, device: str, dataset: str):
    
    loss = 0
    results = []
    model.eval()
    with torch.no_grad():
        for batch_index, (X, y) in tqdm(enumerate(test_dataloader)):
            X_data, y_data = X.to(device), y.to(device)
            y_pred = model(X_data)
            loss += loss_fn(y_pred, y_data).item()
            results.extend(torch.argmax(y_pred, dim=-1) == y_data)
    
    test_accu = sum(results)/len(results)
    test_accu = test_accu.item()
    test_loss = loss/len(results)
    print(dataset + "_loss: " + str(test_loss), end=' ')
    print(dataset + "_accu: " + str(test_accu), end=' ')
    return False

### Train and Track the Model

In [ ]:
# exporti train

config['num_epochs'] = 25

for epoch in tqdm(range(config['num_epochs'])):
    print("epoch: " + str(epoch), end=' ')
    train_fn(res_net, train_dataloader, loss_fn, optimizer, epoch, config['device'])
    test_fn(res_net, valid_dataloader, epoch, config['device'], dataset="valid")    
    print()

### Test and Save the network

In [ ]:
# exporti predict
def predict_fn(model, test_dataloader, device):

    y_pred_list = []
    with torch.no_grad():
        for batch_index, (X, y) in enumerate(test_dataloader):
            X_data, y_data = X.to(device), y.to(device)
            y_pred = model(X_data)
            y_pred_list.append(y_pred)
    return torch.cat(y_pred_list)

In [ ]:
# exporti train
test_fn(res_net, test_dataloader, epoch, config['device'], dataset="test")

In [ ]:
# exporti train
torch.save(res_net.state_dict(), "resnet_cifar10_n3_network.pth")

### Conclusion and Future work